## 1. 데이터 로드

In [173]:
import pandas as pd
import numpy as np
from konlpy.tag import Komoran
import matplotlib.pyplot as plt

In [2]:
purpose = pd.read_csv("../data/수정데이터/용도별 목적대화 데이터.csv")
topic = pd.read_csv("../data/수정데이터/주제별 텍스트 일상 대화 데이터.csv")
sense = pd.read_csv("../data/수정데이터/일반상식 데이터.csv")
info = pd.read_csv("../data/수정데이터/여행정보 데이터.csv")
recommend = pd.read_csv("../data/수정데이터/여행지추천 데이터.csv")

purpose.dropna(inplace=True)
topic.dropna(inplace=True)
sense.dropna(inplace=True)
info.dropna(inplace=True)
recommend.dropna(inplace=True)

In [8]:
print(f'purpose shape: {purpose.shape}')
print(f'topic shape: {topic.shape}')
print(f'sense shape: {sense.shape}')
print(f'info shape: {info.shape}')
print(f'recommend shape: {recommend.shape}')

purpose shape: (225974, 1)
topic shape: (870138, 1)
sense shape: (68538, 3)
info shape: (11400, 3)
recommend shape: (980, 2)


In [9]:
print(purpose.columns)
print(topic.columns)
print(sense.columns)
print(info.columns)
print(recommend.columns)

Index(['text'], dtype='object')
Index(['text'], dtype='object')
Index(['intent', 'question', 'answer'], dtype='object')
Index(['city', 'title', 'label'], dtype='object')
Index(['question', 'answer'], dtype='object')


In [77]:
total_data = list(purpose['text']) + list(topic['text']) + list(sense['question']) + list(sense['answer']) + \
            list(info['city']) + list(info['title']) + list(recommend['question']) + list(recommend['answer'])

In [78]:
len(total_data)

1257948

In [79]:
df = pd.DataFrame({'text': total_data})
df.to_csv('../data/수정데이터/통합데이터.csv', index=False)

## 2. 의도 분류 데이터 생성
* 0: 장소
* 1: 시간
* 2: 기타

In [138]:
place = []
time = []
etc = []
nan = []

In [139]:
for data in total_data:
    place_keywords = ['국내', '여행', '장소', '주소', '위치', '추천', '어디', '근처', '맛집' ] 
    time_keywords = ['봄', '여름', '가을', '겨울', '시간', '기간', '언제', '계획']
    try:
        if any(keyword in data for keyword in place_keywords):
            place.append(data)
        elif any(keyword in data for keyword in time_keywords):
            time.append(data)
        else:
            etc.append(data)
    except:
        nan.append(data)

In [140]:
place_df = pd.DataFrame({'text': place})
place_df['label'] = 0
place_df.head()

,text,label
0,저희가 부산 해운대구에는 주소 에 있는 센타가 있고요,0
1,아니시면 주소 에 있는 작은 분점도 확인되고 있습니다,0
2,주소 이 나을 거 같아요,0
3,그러세요 그러시다면 주소 은 지하철 장산역 십 사 번 출구에 있구요,0
4,아까 주소 얘기 하다가 끊어졌죠 주소 에 있는 주소 에요,0


In [141]:
time_df = pd.DataFrame({'text': time})
time_df['label'] = 1
time_df.tail()

,text,label
43637,강릉어떨까요 청량리역 강릉역 시간 분대 서울역 강릉역 시간 ...,1
43638,답변드립니다 다음은 경주보문단지요 경주는 화엄사등 박 일로도 부족한 많은 유...,1
43639,강릉이요 기차타고 가시면 시 분 무궁화호타시고 주변에 카페하나가 있어요 카페에...,1
43640,통영만빼고는 부산 경주는기차로갈수있고 전라도쪽보다는 부산 이런곳은 교통망도 좋아서 ...,1
43641,통영만빼고는 부산 경주는기차로갈수있고 전라도쪽보다는 부산 이런곳은 교통망도 좋아서 ...,1


In [142]:
etc_df = pd.DataFrame({'text': etc})
etc_df['label'] = 2
etc_df.tail()

,text,label
1174231,군산이야 당연히 바닷가죠 공단과 새만금방조제로 바다가 막혀있기는 하지만 군산은 서...,2
1174232,울릉도 가보세요 기상좋으면 독도까지 갔다 오시고 알아두세요 위 답변은 답변작성자가...,2
1174233,강원도 동해시 괜찮습니다 강릉도 좋구요 알아두세요 위 답변은 답변작성자가 경험과...,2
1174234,여수로가면 볼거리 먹거리 놀거리 좋아요 하멜등대 고소동벽화마을 오동도 자산공원 케이...,2
1174235,강릉 까지가도 되겠네요 강릉 강문해변 안목해변 커피거리 경포호 해변 오죽헌 선교장 ...,2


In [143]:
nan

[]

In [153]:
total_train_data = pd.concat([place_df, time_df, etc_df], axis=0)
total_train_data.reset_index(drop=True, inplace=True)
total_train_data.shape

(1257948, 2)

In [154]:
total_train_data[total_train_data['label'] == 0]

,text,label
0,저희가 부산 해운대구에는 주소 에 있는 센타가 있고요,0
1,아니시면 주소 에 있는 작은 분점도 확인되고 있습니다,0
2,주소 이 나을 거 같아요,0
3,그러세요 그러시다면 주소 은 지하철 장산역 십 사 번 출구에 있구요,0
4,아까 주소 얘기 하다가 끊어졌죠 주소 에 있는 주소 에요,0
...,...,...
40065,문의주신 내용에 대한 답변 드립니다 경기도와 강원도 사이에 위치한 남이섬은 ...,0
40066,안녕하세요 부산에 가볼만한곳도많고 맛집도 너무많아서 고민이 많이되실꺼에요 ㅜㅜ 부...,0
40067,저는 강원도에 안번데기 다녀왔는데 너무 좋았어서 추천드립니다 답변해드리면서 강원도...,0
40068,포항에 일본가옥거리 추천해드릴게요 작은 골목이라서 넓지는 않지만 옛날 생각나는 불...,0


In [155]:
total_train_data[total_train_data['label'] == 1]

,text,label
40070,언제 쯤 연락을 해주셨나요 담당자님이랑요,1
40071,네 확인 감사합니다 혹시 이거 언제 어제 보내셨나요,1
40072,한 몇 시간 된 거 같은데 세네 시간 됐을까,1
40073,두 시간 두세 시간 됐나요,1
40074,지금 송금시킬게요 언제 정도 택배가 와요,1
...,...,...
83707,강릉어떨까요 청량리역 강릉역 시간 분대 서울역 강릉역 시간 ...,1
83708,답변드립니다 다음은 경주보문단지요 경주는 화엄사등 박 일로도 부족한 많은 유...,1
83709,강릉이요 기차타고 가시면 시 분 무궁화호타시고 주변에 카페하나가 있어요 카페에...,1
83710,통영만빼고는 부산 경주는기차로갈수있고 전라도쪽보다는 부산 이런곳은 교통망도 좋아서 ...,1


In [156]:
total_train_data[total_train_data['label'] == 2]

,text,label
83712,반갑습니다 소속 상담사 이름 입니다,2
83713,안내해주신 방법을 시도해 봤는데 안 돼서요,2
83714,그러시다면 두 번째 방법은요,2
83715,휴대폰 명의자 본인이요,2
83716,휴대폰하고 신분증 챙기셔서 저희 서비스센터 쪽으로 방문 하셔야 되세요 고객님,2
...,...,...
1257943,군산이야 당연히 바닷가죠 공단과 새만금방조제로 바다가 막혀있기는 하지만 군산은 서...,2
1257944,울릉도 가보세요 기상좋으면 독도까지 갔다 오시고 알아두세요 위 답변은 답변작성자가...,2
1257945,강원도 동해시 괜찮습니다 강릉도 좋구요 알아두세요 위 답변은 답변작성자가 경험과...,2
1257946,여수로가면 볼거리 먹거리 놀거리 좋아요 하멜등대 고소동벽화마을 오동도 자산공원 케이...,2


In [157]:
total_train_data.to_csv('../data/수정데이터/total_train_data.csv', index=False)

## 3. 적절한 패딩 길이 탐색

In [158]:
total_train_data = pd.read_csv('../data/수정데이터/total_train_data.csv')

In [159]:
tokenizer = Komoran()

In [163]:
data_tokenized = [[token + '/' + POS for token, POS in tokenizer.pos(text)] for text in total_train_data['text']]

In [165]:
exclusion_tags = [
    "JKS", "JKC", "JKG", "JKO", "JKB", "JKV", "JKQ",
    "JX", "JC",
    "SF", "SP", "SS", "SE", "SO",
    "EP", "EF", "EC", "ETN", "ETM",
    "XSN", "XSV", "XSA",
]
def Exclusion_tags(x):
    return x in exclusion_tags

data_list = []
for i in range(len(data_tokenized)):
    temp = []
    for j in range(len(data_tokenized[i])):
        if Exclusion_tags(data_tokenized[i][j].split('/')[1]) is False:
            temp.append(data_tokenized[i][j].split('/')[0])
    data_list.append(temp)

In [169]:
len(data_list)

1257948

In [181]:
num_tokens = [len(sent) for sent in data_list]
num_tokens = np.array(num_tokens)
print('토큰 길이 평균:', num_tokens.mean())
print('토큰 길이 최대:', num_tokens.max())
print('토큰 길이 표준편차:', num_tokens.std())

토큰 길이 평균: 6.450725308200339
토큰 길이 최대: 1984
토큰 길이 표준편차: 9.888866319442958


In [187]:
MAX_SEQ_LEN = 15
def below_len_rate(max_len, data_list):
    cnt = 0
    for s in data_list:
        if (len(s) <= max_len):
            cnt = cnt + 1

    print(f'전체 샘플 중 길이가 {max_len} 이하인 샘플의 비율: {(cnt / len(data_list))}')
below_len_rate(MAX_SEQ_LEN, data_list)

전체 샘플 중 길이가 15 이하인 샘플의 비율: 0.9868945298215824
